# All Results


In [14]:
import numpy as np

import umap

import torch
import json
import os
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

/home/user/understanding_understanding/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def get_layer_activations(internal_reps, layer_idx):
    """
    Get the activations of a specific layer for all problems and all examples.

    internal_reps: list[tuple[tensor]] where internal_reps[question_num][layer_num]
     is a tensor of shape [batch=1, num_tokens, d_model=4096]

    We will return a tensor of shape [num_questions, num_tokens, d_model=4096]
    for the given layer
    """
    return torch.cat([internal_reps[i][layer_idx] for i in range(len(internal_reps))])


## x + y UMAP 20240709

```bash
python3 compute_minute_math_reps.py \
    --output_dir results/xyUMAP20240709 \
    --problem_type xy
```

In [3]:
# Define the directory containing the results
results_dir = 'xyUMAP20240709'

In [5]:
# Load the answers and problems files
answers = np.load(os.path.join(results_dir, 'answers.npy'))
problems = np.load(os.path.join(results_dir, 'problems.npy'))

# Load the internal representations and logits
internal_reps = torch.load(os.path.join(results_dir, 'internal_reps.pt'))
logits = torch.load(os.path.join(results_dir, 'logits.pt'))

# Load the arguments
with open(os.path.join(results_dir, 'args.json'), 'r') as f:
    args = json.load(f)

# Display the shapes and types of the loaded data
print("Answers shape:", answers.shape)
print("Problems shape:", problems.shape)
print("Internal Reps type:", type(internal_reps))
print("Internal Reps length:", len(internal_reps))
print("Logits type:", type(logits))
print("Logits length:", len(logits))

# Let's inspect the internal representations and logits a bit more closely
print("Example internal representation shape:", internal_reps[0][0][0].shape)
print("Example logits shape:", logits[0].shape)

# Display some examples from the loaded data
print("\nExample answers:", answers[0])
print("\nExample problems:", problems[0])
print("\nExample internal representation:", internal_reps[0][0][0])
print("\nExample logits:", logits[0])



/home/user/understanding_understanding/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Answers shape: (100, 1)
Problems shape: (100, 17)
Internal Reps type: <class 'list'>
Internal Reps length: 100
Logits type: <class 'torch.Tensor'>
Logits length: 100
Example internal representation shape: torch.Size([17, 4096])
Example logits shape: torch.Size([17, 128256])

Example answers: [868]

Example problems: [128000     87    284    220    975     11    379    284    220     16
     26   9093    865    489    379    284    220]

Example internal representation: tensor([[-8.2970e-05,  2.5749e-04, -2.4605e-04,  ..., -3.2425e-04,
         -2.1553e-04,  4.7112e-04],
        [-2.0752e-03, -1.4038e-03,  6.1035e-03,  ...,  2.2888e-03,
          6.1951e-03,  1.1414e-02],
        [ 1.0376e-03, -6.8054e-03,  6.2943e-04,  ...,  2.5482e-03,
         -8.3618e-03, -8.8501e-03],
        ...,
        [ 3.3569e-03, -3.3760e-04,  2.4719e-03,  ..., -8.3008e-03,
          3.2654e-03, -7.3242e-03],
        [ 1.0376e-03, -6.8054e-03,  6.2943e-04,  ...,  2.5482e-03,
         -8.3618e-03, -8.8501e-03]

### Check correctness of model predictions

In [30]:
# compute the actual token-wise answers
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(args['model_name'])

# find the argmax over the logits for each 
# problem to get the predicted class

# logits has shape [num_questions, num_tokens, vocab_size]

final_logits = logits[:, -1, :]

# take the argmax over the final dim
predicted_class = torch.argmax(final_logits, dim=1)

# decode each prediction individually
predicted_class_str = [tokenizer.decode(i) for i in predicted_class]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [36]:
str_answers = [tokenizer.decode(i) for i in answers[:, 0]]

str_problems = []
for i in range(problems.shape[0]):
    str_problems.append(tokenizer.decode(problems[i, :]).replace('\n', ' '))

num_correct = 0
for i in range(len(str_answers)):
    if str_answers[i] == predicted_class_str[i]:
        num_correct += 1

num_correct / len(str_answers)

1.0

In [39]:
str_problems

['<|begin_of_text|>x = 14, y = 1; therefore x + y = ',
 '<|begin_of_text|>x = 7, y = 9; therefore x + y = ',
 '<|begin_of_text|>x = 7, y = 10; therefore x + y = ',
 '<|begin_of_text|>x = 7, y = 11; therefore x + y = ',
 '<|begin_of_text|>x = 13, y = 6; therefore x + y = ',
 '<|begin_of_text|>x = 13, y = 7; therefore x + y = ',
 '<|begin_of_text|>x = 8, y = 13; therefore x + y = ',
 '<|begin_of_text|>x = 8, y = 14; therefore x + y = ',
 '<|begin_of_text|>x = 6, y = 17; therefore x + y = ',
 '<|begin_of_text|>x = 4, y = 20; therefore x + y = ',
 '<|begin_of_text|>x = 6, y = 19; therefore x + y = ',
 '<|begin_of_text|>x = 4, y = 11; therefore x + y = ',
 '<|begin_of_text|>x = 7, y = 9; therefore x + y = ',
 '<|begin_of_text|>x = 7, y = 10; therefore x + y = ',
 '<|begin_of_text|>x = 6, y = 12; therefore x + y = ',
 '<|begin_of_text|>x = 6, y = 13; therefore x + y = ',
 '<|begin_of_text|>x = 20, y = 0; therefore x + y = ',
 '<|begin_of_text|>x = 1, y = 20; therefore x + y = ',
 '<|begin_of

In [38]:
# make a set of integer-valued answers
int_answers = [int(a) for a in str_answers]

In [6]:
class_ids = torch.tensor(answers[:, 0])
class_ids.shape

torch.Size([100])

In [27]:

def get_umap(layer_reps):
    """ Compute the UMAP of layer_reps, which is a tensor of shape [num_examples, num_tokens, token_dim]
    """
    print("Layer reps shape:", layer_reps.shape)
    print("Layer reps has type:", type(layer_reps))
    
    # Convert to numpy if it's a torch tensor
    if torch.is_tensor(layer_reps):
        layer_reps = layer_reps.detach().cpu().numpy()
    
    # Flatten the last two dimensions
    num_examples, num_tokens, token_dim = layer_reps.shape
    flattened_reps = layer_reps.reshape(num_examples, num_tokens * token_dim)
    
    # Apply UMAP
    reducer = umap.UMAP(random_state=42)
    umap_embedding = reducer.fit_transform(flattened_reps)
    # print("Output of reduce transform: ", umap_embedding)
    
    return umap_embedding


In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# iterate through each layer
num_layers = 33
purity_by_layer = []
for layer_num in range(num_layers):
    layer_activations = get_layer_activations(internal_reps, layer_num)
    umap_embedding = get_umap(layer_activations)
    
    # Create Plotly figure
    fig = make_subplots(rows=1, cols=1)
    
    # Add scatter plot
    scatter = go.Scatter(
        x=umap_embedding[:, 0],
        y=umap_embedding[:, 1],
        mode='markers',
        marker=dict(
            size=8,
            color=int_answers,
            colorscale='Viridis',
            showscale=True
        ),
        text=[f"Problem: {prob}<br>Answer: {ans}<br>Predicted: {pred}" 
              for prob, ans, pred in zip(str_problems, str_answers, predicted_class_str)],
        hoverinfo='text'
    )
    
    fig.add_trace(scatter)
    
    # Update layout
    fig.update_layout(
        title=f'UMAP for Layer {layer_num}',
        xaxis_title='UMAP 1',
        yaxis_title='UMAP 2',
        width=1000,
        height=800
    )
    
    # Save as interactive HTML
    pio.write_html(fig, file=f'{results_dir}/umap_layer_{layer_num}.html')
    
    # Save as static PNG
    pio.write_image(fig, file=f'{results_dir}/umap_layer_{layer_num}.png')
    
    # Clear the figure to free up memory
    fig.data = []
    fig.layout = {}

In [29]:
# iterate through each layer
num_layers = 33
purity_by_layer = []
for layer_num in range(num_layers):
    layer_activations = get_layer_activations(internal_reps, layer_num)
    umap_embedding = get_umap(layer_activations)
    
    # Plot UMAP
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(umap_embedding[:, 0], umap_embedding[:, 1], c=int_answers, cmap='viridis')
    plt.colorbar(scatter)
    plt.title(f'UMAP for Layer {layer_num}')
    plt.savefig(f'{results_dir}/umap_layer_{layer_num}.png')
    plt.close()
    

Layer reps shape: torch.Size([100, 17, 4096])
Layer reps has type: <class 'torch.Tensor'>


/home/user/understanding_understanding/venv/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Umap embedding:  [[ 1.9839981  16.961124  ]
 [ 0.3122297   9.53469   ]
 [ 0.9433634  10.770165  ]
 [ 1.1211493  10.786196  ]
 [ 2.4040287  13.82416   ]
 [ 2.2168703  13.971267  ]
 [ 1.913667   10.3012295 ]
 [ 2.8886292  10.138913  ]
 [ 1.5689002   7.715608  ]
 [ 2.207153    7.3802876 ]
 [ 2.2237859   8.398431  ]
 [ 1.3722275  10.850339  ]
 [ 0.40785846  9.764785  ]
 [ 0.88500494 10.697695  ]
 [ 1.4250427   9.5749655 ]
 [ 1.7974268  10.063754  ]
 [ 1.0985771  16.595497  ]
 [ 2.6004758   7.653791  ]
 [ 2.2104053   9.451104  ]
 [ 2.0350487  14.710997  ]
 [ 2.8337095  10.842353  ]
 [ 2.3271286   6.8281927 ]
 [ 2.4520853  12.784485  ]
 [ 2.9879336   9.718846  ]
 [ 1.8483367  13.663787  ]
 [ 2.1277366  13.402206  ]
 [ 1.2422247  11.150928  ]
 [ 0.976738   16.649273  ]
 [ 2.1799304  15.495153  ]
 [ 2.6964252   7.093963  ]
 [ 2.7360554  10.192565  ]
 [ 2.3617494   9.790044  ]
 [ 2.8682053  10.849595  ]
 [ 0.44039902  9.421431  ]
 [ 1.9551414  12.680347  ]
 [ 2.314585   13.572343  ]
 [ 1.431854

/home/user/understanding_understanding/venv/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Umap embedding:  [[8.92883   4.262426 ]
 [1.9211676 4.6988854]
 [2.7462568 4.780979 ]
 [3.045099  4.581351 ]
 [6.167577  5.0481687]
 [6.2603836 4.7583265]
 [3.4721048 6.45933  ]
 [3.226765  5.721777 ]
 [1.0584984 6.89254  ]
 [1.3867323 8.424942 ]
 [1.3220412 6.672735 ]
 [3.2467744 4.8445787]
 [1.9135064 4.766876 ]
 [2.7001674 4.7262735]
 [2.389038  6.294583 ]
 [2.9053516 6.423448 ]
 [9.525268  4.9642334]
 [1.7609824 8.3247595]
 [2.6522567 5.5558095]
 [6.533649  4.5737095]
 [3.9652257 5.5745473]
 [1.3372599 8.768498 ]
 [5.6703725 4.8518524]
 [2.413286  7.207587 ]
 [6.5276346 5.1453485]
 [5.956752  5.3294144]
 [3.3969443 4.707712 ]
 [9.55446   5.086205 ]
 [7.6127934 4.8274617]
 [1.768728  8.6891365]
 [3.379949  6.008908 ]
 [3.4536498 6.0294456]
 [4.234451  5.4601283]
 [1.737929  4.8520794]
 [4.9634147 5.062974 ]
 [5.950555  4.8669295]
 [9.241321  4.752397 ]
 [5.9699936 5.0689263]
 [1.910411  7.425026 ]
 [1.8166504 7.698469 ]
 [1.7329148 6.5090446]
 [1.8539102 6.0333867]
 [2.0795627 8.848

...

Umap embedding:  [[ 7.4036636   8.140168  ]
 [10.597338    5.726423  ]
 [10.680693    5.854545  ]
 [10.740029    5.9395623 ]
 [ 7.3959312   6.8725085 ]
 [ 7.325092    6.885338  ]
 [ 9.850405    4.7387843 ]
 [ 9.9072895   4.7636375 ]
 [10.204758    3.7151911 ]
 [ 8.313418    2.4326758 ]
 [10.080065    3.5154645 ]
 [ 8.720862    2.705513  ]
 [10.531863    5.71168   ]
 [10.8147545   5.8849554 ]
 [ 9.819933    3.6401944 ]
 [10.015019    3.771237  ]
 [ 6.793221    8.055877  ]
 [10.138371    0.5332522 ]
 [10.697302    5.2434497 ]
 [ 6.7001634   6.648645  ]
 [ 8.854769    5.827485  ]
 [ 8.453557    2.5900056 ]
 [ 8.7791605   4.3415003 ]
 [ 9.923685   -1.1703689 ]
 [ 7.9183307   6.488223  ]
 [ 8.092957    6.6543126 ]
 [ 9.81375     4.896343  ]
 [ 6.4520764   7.9268374 ]
 [ 6.300093    7.077192  ]
 [10.318333    0.2978913 ]
 [ 8.896141    4.7270536 ]
 [ 8.820153    4.5968766 ]
 [ 8.308542    7.195973  ]
 [ 9.817717    3.4583225 ]
 [ 9.886882    5.115875  ]
 [ 8.959178    6.041656  ]
 [ 5.385125

/home/user/understanding_understanding/venv/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Umap embedding:  [[11.49564    9.205316 ]
 [ 9.532836   2.4747086]
 [ 9.365758   2.5164628]
 [ 9.229102   2.352812 ]
 [10.978217   8.045194 ]
 [11.313697   8.017826 ]
 [ 9.665478   4.199231 ]
 [ 9.506754   4.5700154]
 [ 8.38721    5.855896 ]
 [11.462758   4.423329 ]
 [ 8.496696   6.0091767]
 [11.20038    4.2422266]
 [ 9.274847   2.3943586]
 [ 9.259849   2.3246915]
 [ 8.660717   6.0024195]
 [ 8.553341   6.06893  ]
 [12.097859   9.052423 ]
 [ 7.2061334  2.4364128]
 [ 9.42589    2.846973 ]
 [12.079398   7.8451233]
 [10.272878   6.3513823]
 [11.6027     4.5547504]
 [ 9.579589   5.571949 ]
 [ 6.1836233  1.2870212]
 [10.9338455  7.372583 ]
 [11.325728   7.372616 ]
 [ 9.402389   4.216576 ]
 [12.115184   9.231174 ]
 [12.780831   7.864177 ]
 [ 7.000178   2.308054 ]
 [ 9.866408   5.4950347]
 [ 9.8869095  5.5471973]
 [10.368287   7.531251 ]
 [ 8.704128   6.0092897]
 [ 9.429215   4.07089  ]
 [10.444132   6.5961585]
 [13.167769   8.9751835]
 [11.037955   7.9627037]
 [ 5.789366   1.6845284]
 [ 6.117

/home/user/understanding_understanding/venv/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Umap embedding:  [[ 9.3070135  8.094118 ]
 [ 4.142883   6.976472 ]
 [ 4.231204   6.8490925]
 [ 4.0301085  6.8636723]
 [ 7.3634267  8.400935 ]
 [ 7.2656417  8.517849 ]
 [ 3.9076314  8.391752 ]
 [ 3.6964405  8.396708 ]
 [ 1.8350085  8.88178  ]
 [ 2.6142392 14.164428 ]
 [ 1.7662842  8.840408 ]
 [ 2.986344  14.282887 ]
 [ 4.2079573  6.663436 ]
 [ 4.0185294  6.6614237]
 [ 2.00921    9.240819 ]
 [ 1.6436301  9.117401 ]
 [ 9.786003   7.4193454]
 [ 5.5554404 15.810875 ]
 [ 3.9275765  7.0598383]
 [ 7.703227   8.168382 ]
 [ 5.5428805  7.768938 ]
 [ 2.6547067 14.114721 ]
 [ 3.8581698  9.21844  ]
 [ 5.09662   17.182373 ]
 [ 6.6070933  8.103585 ]
 [ 6.5416036  8.224497 ]
 [ 4.053017   8.251582 ]
 [ 9.5624695  7.638177 ]
 [ 8.350115   8.390437 ]
 [ 5.707492  15.831445 ]
 [ 3.7975733  8.962038 ]
 [ 3.7844236  9.067569 ]
 [ 6.2984447  8.804098 ]
 [ 2.2624214  9.065834 ]
 [ 4.1853805  8.48276  ]
 [ 5.61702    7.6448336]
 [ 9.214404   7.2557907]
 [ 7.3539124  8.237324 ]
 [ 4.977115  16.707798 ]
 [ 5.350

In [13]:
type(answers)

numpy.ndarray